In [1]:
%matplotlib inline

import os

# let's not pollute this blog post with warnings
from warnings import filterwarnings
filterwarnings('ignore')

import keras
import numpy as np
import pandas as pd
import skvideo.io as skv
from tqdm import tqdm

Using TensorFlow backend.


In [2]:
# load the data
run = '-mobilenet-overfit-micro'
labelpath = os.path.join('train_labels.csv')
train_labels = pd.read_csv(labelpath, index_col='filename')

In [3]:
train_labels.head()

,bird,blank,cattle,chimpanzee,elephant,forest buffalo,gorilla,hippopotamus,human,hyena,...,other (primate),pangolin,porcupine,reptile,rodent,small antelope,small cat,wild dog,duiker,hog
filename,,,,,,,,,,,,,,,,,,,,,
000libDc84.mp4,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
003TeGtbkD.mp4,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
006jFoesFi.mp4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
008uxqP8IN.mp4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
0094UxdyyZ.mp4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [4]:
train_labels.info()

<class 'pandas.core.frame.DataFrame'>
Index: 204130 entries, 000libDc84.mp4 to zzzu2lK8bC.mp4
Data columns (total 24 columns):
bird                   204130 non-null float64
blank                  204130 non-null float64
cattle                 204130 non-null float64
chimpanzee             204130 non-null float64
elephant               204130 non-null float64
forest buffalo         204130 non-null float64
gorilla                204130 non-null float64
hippopotamus           204130 non-null float64
human                  204130 non-null float64
hyena                  204130 non-null float64
large ungulate         204130 non-null float64
leopard                204130 non-null float64
lion                   204130 non-null float64
other (non-primate)    204130 non-null float64
other (primate)        204130 non-null float64
pangolin               204130 non-null float64
porcupine              204130 non-null float64
reptile                204130 non-null float64
rodent                 2041

In [5]:
train_labels.sum(axis=0).sort_values(ascending=False)

blank                  122270.0
duiker                  21601.0
other (primate)         20453.0
human                   20034.0
chimpanzee               5045.0
hog                      4650.0
rodent                   2911.0
bird                     2386.0
other (non-primate)      1883.0
elephant                 1085.0
porcupine                 569.0
cattle                    372.0
small antelope            273.0
large ungulate            224.0
leopard                   209.0
hippopotamus              175.0
gorilla                   174.0
small cat                  79.0
pangolin                   63.0
wild dog                   21.0
hyena                      10.0
forest buffalo              9.0
reptile                     8.0
lion                        2.0
dtype: float64

In [6]:
(train_labels.sum(axis=1) > 1).sum()

374

In [7]:
from primatrix_dataset_utils_overfit import Dataset

In [8]:
datapath = os.path.join('.')
whichset = run.split('-')[-1]
redframes = whichset == 'nano'
data = Dataset(datapath=datapath, 
               dataset_type='micro-overfit',
               reduce_frames=False, 
               batch_size=32, 
               test=False)

['000libDc84.mp4' '003TeGtbkD.mp4' '006jFoesFi.mp4' '008uxqP8IN.mp4'
 '0094UxdyyZ.mp4' '009Bm4R6ke.mp4' '00Cw71Q7mp.mp4' '00DAdApXV5.mp4'
 '00EcW4Z4Iy.mp4' '00GcOZpJIr.mp4' '00GnlMwGm0.mp4' '00IBE38OWj.mp4'
 '00JRe5ZGg7.mp4' '00JiLU8uBe.mp4' '00KLCPKkFC.mp4' '00KPNQcfrv.mp4'
 '00M4lnQV16.mp4' '00MqYQh31d.mp4' '00PkBYslCh.mp4' '00QuBrCWLL.mp4'
 '00SDd7SSFf.mp4' '00UejWIbCh.mp4' '00UhxKIrHG.mp4' '00VE5WBpwZ.mp4'
 '00YqPojkvr.mp4' '00ZthvsuWW.mp4' '00aoQMT1EU.mp4' '00b8iiWvAJ.mp4'
 '00e8EQeEu5.mp4' '00eLJaaQOF.mp4' '00fGoKSs4A.mp4' '00iW7nBPxy.mp4']
[[ 0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.  0.  0.  0.  0.]
 [ 0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.  0.  0.  1.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.  0.  0.  0.  1.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  

In [9]:
data.num_frames

30

In [10]:
data.batch_size

32

In [11]:
data.num_samples

32

In [12]:
from tensorflow.python.client import device_lib

def get_available_gpus():
    local_device_protos = device_lib.list_local_devices()
    return [x.name for x in local_device_protos if x.device_type == 'GPU']

print(get_available_gpus())

['/gpu:0']


In [13]:
from keras.models import Sequential
from keras.layers import TimeDistributed, GlobalAveragePooling2D, Activation, Dense, Input
from keras.layers import BatchNormalization, Conv2D, MaxPooling2D, AveragePooling2D, GlobalAveragePooling2D
from keras.layers import GlobalMaxPooling2D
from keras.layers.recurrent import LSTM
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras.layers import Dropout, Flatten
from keras.layers import concatenate
from keras import regularizers
from keras import initializers
from keras import constraints
from keras.models import Model
# Backend
from keras import backend as K
# Utils
from keras.utils.layer_utils import convert_all_kernels_in_model
from keras.utils.data_utils import get_file
from keras.utils import conv_utils
from keras.engine.topology import get_source_inputs
from keras.engine import InputSpec


def relu6(x):
    return K.relu(x, max_value=6)

class DepthwiseConv2D(Conv2D):
    """Depthwise separable 2D convolution.
    Depthwise Separable convolutions consists in performing
    just the first step in a depthwise spatial convolution
    (which acts on each input channel separately).
    The `depth_multiplier` argument controls how many
    output channels are generated per input channel in the depthwise step.
    # Arguments
        kernel_size: An integer or tuple/list of 2 integers, specifying the
            width and height of the 2D convolution window.
            Can be a single integer to specify the same value for
            all spatial dimensions.
        strides: An integer or tuple/list of 2 integers,
            specifying the strides of the convolution along the width and height.
            Can be a single integer to specify the same value for
            all spatial dimensions.
            Specifying any stride value != 1 is incompatible with specifying
            any `dilation_rate` value != 1.
        padding: one of `'valid'` or `'same'` (case-insensitive).
        depth_multiplier: The number of depthwise convolution output channels
            for each input channel.
            The total number of depthwise convolution output
            channels will be equal to `filters_in * depth_multiplier`.
        data_format: A string,
            one of `channels_last` (default) or `channels_first`.
            The ordering of the dimensions in the inputs.
            `channels_last` corresponds to inputs with shape
            `(batch, height, width, channels)` while `channels_first`
            corresponds to inputs with shape
            `(batch, channels, height, width)`.
            It defaults to the `image_data_format` value found in your
            Keras config file at `~/.keras/keras.json`.
            If you never set it, then it will be 'channels_last'.
        activation: Activation function to use
            (see [activations](../activations.md)).
            If you don't specify anything, no activation is applied
            (ie. 'linear' activation: `a(x) = x`).
        use_bias: Boolean, whether the layer uses a bias vector.
        depthwise_initializer: Initializer for the depthwise kernel matrix
            (see [initializers](../initializers.md)).
        bias_initializer: Initializer for the bias vector
            (see [initializers](../initializers.md)).
        depthwise_regularizer: Regularizer function applied to
            the depthwise kernel matrix
            (see [regularizer](../regularizers.md)).
        bias_regularizer: Regularizer function applied to the bias vector
            (see [regularizer](../regularizers.md)).
        activity_regularizer: Regularizer function applied to
            the output of the layer (its 'activation').
            (see [regularizer](../regularizers.md)).
        depthwise_constraint: Constraint function applied to
            the depthwise kernel matrix
            (see [constraints](../constraints.md)).
        bias_constraint: Constraint function applied to the bias vector
            (see [constraints](../constraints.md)).
    # Input shape
        4D tensor with shape:
        `[batch, channels, rows, cols]` if data_format='channels_first'
        or 4D tensor with shape:
        `[batch, rows, cols, channels]` if data_format='channels_last'.
    # Output shape
        4D tensor with shape:
        `[batch, filters, new_rows, new_cols]` if data_format='channels_first'
        or 4D tensor with shape:
        `[batch, new_rows, new_cols, filters]` if data_format='channels_last'.
        `rows` and `cols` values might have changed due to padding.
    """

    def __init__(self,
                 kernel_size,
                 strides=(1, 1),
                 padding='valid',
                 depth_multiplier=1,
                 data_format=None,
                 activation=None,
                 use_bias=True,
                 depthwise_initializer='he_normal',
                 bias_initializer='zeros',
                 depthwise_regularizer=None,
                 bias_regularizer=None,
                 activity_regularizer=None,
                 depthwise_constraint=None,
                 bias_constraint=None,
                 **kwargs):
        super(DepthwiseConv2D, self).__init__(
            filters=None,
            kernel_size=kernel_size,
            strides=strides,
            padding=padding,
            data_format=data_format,
            activation=activation,
            use_bias=use_bias,
            bias_regularizer=bias_regularizer,
            activity_regularizer=activity_regularizer,
            bias_constraint=bias_constraint,
            **kwargs)
        self.depth_multiplier = depth_multiplier
        self.depthwise_initializer = initializers.get(depthwise_initializer)
        self.depthwise_regularizer = regularizers.get(depthwise_regularizer)
        self.depthwise_constraint = constraints.get(depthwise_constraint)
        self.bias_initializer = initializers.get(bias_initializer)

    def build(self, input_shape):
        if len(input_shape) < 4:
            raise ValueError('Inputs to `DepthwiseConv2D` should have rank 4. '
                             'Received input shape:', str(input_shape))
        if self.data_format == 'channels_first':
            channel_axis = 1
        else:
            channel_axis = 3
        if input_shape[channel_axis] is None:
            raise ValueError('The channel dimension of the inputs to '
                             '`DepthwiseConv2D` '
                             'should be defined. Found `None`.')
        input_dim = int(input_shape[channel_axis])
        depthwise_kernel_shape = (self.kernel_size[0],
                                  self.kernel_size[1],
                                  input_dim,
                                  self.depth_multiplier)

        self.depthwise_kernel = self.add_weight(
            shape=depthwise_kernel_shape,
            initializer=self.depthwise_initializer,
            name='depthwise_kernel',
            regularizer=self.depthwise_regularizer,
            constraint=self.depthwise_constraint)

        if self.use_bias:
            self.bias = self.add_weight(shape=(input_dim * self.depth_multiplier,),
                                        initializer=self.bias_initializer,
                                        name='bias',
                                        regularizer=self.bias_regularizer,
                                        constraint=self.bias_constraint)
        else:
            self.bias = None
        # Set input spec.
        self.input_spec = InputSpec(ndim=4, axes={channel_axis: input_dim})
        self.built = True

    def call(self, inputs, training=None):
        outputs = K.depthwise_conv2d(
            inputs,
            self.depthwise_kernel,
            strides=self.strides,
            padding=self.padding,
            dilation_rate=self.dilation_rate,
            data_format=self.data_format)

        if self.bias:
            outputs = K.bias_add(
                outputs,
                self.bias,
                data_format=self.data_format)

        if self.activation is not None:
            return self.activation(outputs)

        return outputs

    def compute_output_shape(self, input_shape):
        if self.data_format == 'channels_first':
            rows = input_shape[2]
            cols = input_shape[3]
            out_filters = input_shape[1] * self.depth_multiplier
        elif self.data_format == 'channels_last':
            rows = input_shape[1]
            cols = input_shape[2]
            out_filters = input_shape[3] * self.depth_multiplier

        rows = conv_utils.conv_output_length(rows, self.kernel_size[0],
                                             self.padding,
                                             self.strides[0])
        cols = conv_utils.conv_output_length(cols, self.kernel_size[1],
                                             self.padding,
                                             self.strides[1])

        if self.data_format == 'channels_first':
            return (input_shape[0], out_filters, rows, cols)
        elif self.data_format == 'channels_last':
            return (input_shape[0], rows, cols, out_filters)

    def get_config(self):
        config = super(DepthwiseConv2D, self).get_config()
        config.pop('filters')
        config.pop('kernel_initializer')
        config.pop('kernel_regularizer')
        config.pop('kernel_constraint')
        config['depth_multiplier'] = self.depth_multiplier
        config['depthwise_initializer'] = initializers.serialize(self.depthwise_initializer)
        config['depthwise_regularizer'] = regularizers.serialize(self.depthwise_regularizer)
        config['depthwise_constraint'] = constraints.serialize(self.depthwise_constraint)
        return config


def MobileNet(input_shape=None,
              alpha=1.0,
              depth_multiplier=1,
              dropout=0.,
              include_top=True,
              input_tensor=None,
              pooling=None,
              classes=24):
    """Instantiates the MobileNet architecture.
    Note that only TensorFlow is supported for now,
    therefore it only works with the data format
    `image_data_format='channels_last'` in your Keras config
    at `~/.keras/keras.json`.
    To load a MobileNet model via `load_model`, import the custom
    objects `relu6` and `DepthwiseConv2D` and pass them to the
    `custom_objects` parameter.
    E.g.
    model = load_model('mobilenet.h5', custom_objects={
                       'relu6': mobilenet.relu6,
                       'DepthwiseConv2D': mobilenet.DepthwiseConv2D})
    # Arguments
        input_shape: optional shape tuple, only to be specified
            if `include_top` is False (otherwise the input shape
            has to be `(224, 224, 3)` (with `channels_last` data format)
            or (3, 224, 224) (with `channels_first` data format).
            It should have exactly 3 inputs channels,
            and width and height should be no smaller than 32.
            E.g. `(200, 200, 3)` would be one valid value.
        alpha: controls the width of the network.
            - If `alpha` < 1.0, proportionally decreases the number
                of filters in each layer.
            - If `alpha` > 1.0, proportionally increases the number
                of filters in each layer.
            - If `alpha` = 1, default number of filters from the paper
                 are used at each layer.
        depth_multiplier: depth multiplier for depthwise convolution
            (also called the resolution multiplier)
        dropout: dropout rate
        include_top: whether to include the fully-connected
            layer at the top of the network.
        weights: one of `None` (random initialization),
              'imagenet' (pre-training on ImageNet),
              or the path to the weights file to be loaded.
        input_tensor: optional Keras tensor (i.e. output of
            `layers.Input()`)
            to use as image input for the model.
        pooling: Optional pooling mode for feature extraction
            when `include_top` is `False`.
            - `None` means that the output of the model
                will be the 4D tensor output of the
                last convolutional layer.
            - `avg` means that global average pooling
                will be applied to the output of the
                last convolutional layer, and thus
                the output of the model will be a
                2D tensor.
            - `max` means that global max pooling will
                be applied.
        classes: optional number of classes to classify images
            into, only to be specified if `include_top` is True, and
            if no `weights` argument is specified.
    # Returns
        A Keras model instance.
    # Raises
        ValueError: in case of invalid argument for `weights`,
            or invalid input shape.
        RuntimeError: If attempting to run this model with a
            backend that does not support separable convolutions.
    """
    default_size = 64

    input_shape = (data.num_frames, data.width, data.height, 3)
    row_axis, col_axis = (1, 2)
    rows = input_shape[row_axis]
    cols = input_shape[col_axis]
    inputs = Input(shape=input_shape)
    x = BatchNormalization(axis=-1)(inputs)  

    x = _conv_block(x, 32, alpha, strides=(2, 2))
    x = _depthwise_conv_block(x, 64, alpha, depth_multiplier, block_id=1)

    x = _depthwise_conv_block(x, 128, alpha, depth_multiplier,
                              strides=(2, 2), block_id=2)
    x = _depthwise_conv_block(x, 128, alpha, depth_multiplier, block_id=3)

    x = _depthwise_conv_block(x, 256, alpha, depth_multiplier,
                              strides=(2, 2), block_id=4)
    x = _depthwise_conv_block(x, 256, alpha, depth_multiplier, block_id=5)

    x = _depthwise_conv_block(x, 512, alpha, depth_multiplier,
                              strides=(2, 2), block_id=6)
    x = _depthwise_conv_block(x, 512, alpha, depth_multiplier, block_id=7)
    x = _depthwise_conv_block(x, 512, alpha, depth_multiplier, block_id=8)
    x = _depthwise_conv_block(x, 512, alpha, depth_multiplier, block_id=9)
    x = _depthwise_conv_block(x, 512, alpha, depth_multiplier, block_id=10)
    x = _depthwise_conv_block(x, 512, alpha, depth_multiplier, block_id=11)

    x = _depthwise_conv_block(x, 1024, alpha, depth_multiplier,
                              strides=(2, 2), block_id=12)
    x = _depthwise_conv_block(x, 1024, alpha, depth_multiplier, block_id=13)

    if include_top:
        x = TimeDistributed(Flatten())(x)
        x = LSTM(256, return_sequences=False)(x)        
        x = Dropout(dropout)(x)        
        x = Dense(units=classes, activation='softmax')(x)

    # Create model.
    model = Model(inputs, x, name='mobilenet_%0.2f_%s' % (alpha, rows))

    return model


def _conv_block(inputs, filters, alpha, kernel=(3, 3), strides=(1, 1)):
    """Adds an initial convolution layer (with batch normalization and relu6).
    # Arguments
        inputs: Input tensor of shape `(rows, cols, 3)`
            (with `channels_last` data format) or
            (3, rows, cols) (with `channels_first` data format).
            It should have exactly 3 inputs channels,
            and width and height should be no smaller than 32.
            E.g. `(224, 224, 3)` would be one valid value.
        filters: Integer, the dimensionality of the output space
            (i.e. the number output of filters in the convolution).
        alpha: controls the width of the network.
            - If `alpha` < 1.0, proportionally decreases the number
                of filters in each layer.
            - If `alpha` > 1.0, proportionally increases the number
                of filters in each layer.
            - If `alpha` = 1, default number of filters from the paper
                 are used at each layer.
        kernel: An integer or tuple/list of 2 integers, specifying the
            width and height of the 2D convolution window.
            Can be a single integer to specify the same value for
            all spatial dimensions.
        strides: An integer or tuple/list of 2 integers,
            specifying the strides of the convolution along the width and height.
            Can be a single integer to specify the same value for
            all spatial dimensions.
            Specifying any stride value != 1 is incompatible with specifying
            any `dilation_rate` value != 1.
    # Input shape
        4D tensor with shape:
        `(samples, channels, rows, cols)` if data_format='channels_first'
        or 4D tensor with shape:
        `(samples, rows, cols, channels)` if data_format='channels_last'.
    # Output shape
        4D tensor with shape:
        `(samples, filters, new_rows, new_cols)` if data_format='channels_first'
        or 4D tensor with shape:
        `(samples, new_rows, new_cols, filters)` if data_format='channels_last'.
        `rows` and `cols` values might have changed due to stride.
    # Returns
        Output tensor of block.
    """
    channel_axis = -1
    filters = int(filters * alpha)
    x = TimeDistributed(Conv2D(filters, kernel,
               padding='same',
               use_bias=False,
               strides=strides,
               name='conv1',
               kernel_initializer=initializers.he_normal()))(inputs)
    x = TimeDistributed(BatchNormalization(axis=channel_axis, name='conv1_bn'))(x)
    return Activation(relu6, name='conv1_relu')(x)


def _depthwise_conv_block(inputs, pointwise_conv_filters, alpha,
                          depth_multiplier=1, strides=(1, 1), block_id=1):
    """Adds a depthwise convolution block.
    A depthwise convolution block consists of a depthwise conv,
    batch normalization, relu6, pointwise convolution,
    batch normalization and relu6 activation.
    # Arguments
        inputs: Input tensor of shape `(rows, cols, channels)`
            (with `channels_last` data format) or
            (channels, rows, cols) (with `channels_first` data format).
        pointwise_conv_filters: Integer, the dimensionality of the output space
            (i.e. the number output of filters in the pointwise convolution).
        alpha: controls the width of the network.
            - If `alpha` < 1.0, proportionally decreases the number
                of filters in each layer.
            - If `alpha` > 1.0, proportionally increases the number
                of filters in each layer.
            - If `alpha` = 1, default number of filters from the paper
                 are used at each layer.
        depth_multiplier: The number of depthwise convolution output channels
            for each input channel.
            The total number of depthwise convolution output
            channels will be equal to `filters_in * depth_multiplier`.
        strides: An integer or tuple/list of 2 integers,
            specifying the strides of the convolution along the width and height.
            Can be a single integer to specify the same value for
            all spatial dimensions.
            Specifying any stride value != 1 is incompatible with specifying
            any `dilation_rate` value != 1.
        block_id: Integer, a unique identification designating the block number.
    # Input shape
        4D tensor with shape:
        `(batch, channels, rows, cols)` if data_format='channels_first'
        or 4D tensor with shape:
        `(batch, rows, cols, channels)` if data_format='channels_last'.
    # Output shape
        4D tensor with shape:
        `(batch, filters, new_rows, new_cols)` if data_format='channels_first'
        or 4D tensor with shape:
        `(batch, new_rows, new_cols, filters)` if data_format='channels_last'.
        `rows` and `cols` values might have changed due to stride.
    # Returns
        Output tensor of block.
    """
    channel_axis = -1
    pointwise_conv_filters = int(pointwise_conv_filters * alpha)

    x = TimeDistributed(DepthwiseConv2D((3, 3),
                        padding='same',
                        depth_multiplier=depth_multiplier,
                        strides=strides,
                        use_bias=False,
                        name='conv_dw_%d' % block_id))(inputs)
    x = TimeDistributed(BatchNormalization(axis=channel_axis, name='conv_dw_%d_bn' % block_id))(x)
    x = Activation(relu6, name='conv_dw_%d_relu' % block_id)(x)

    x = TimeDistributed(Conv2D(pointwise_conv_filters, (1, 1),
               padding='same',
               use_bias=False,
               strides=(1, 1),
               name='conv_pw_%d' % block_id,
               kernel_initializer=initializers.he_normal()))(x)
    x = TimeDistributed(BatchNormalization(axis=channel_axis, name='conv_pw_%d_bn' % block_id))(x)
    return Activation(relu6, name='conv_pw_%d_relu' % block_id)(x)

In [14]:
# instantiate model
#model = VGG19()
model = MobileNet(dropout=0.2)


# classifier with sigmoid activation for multilabel

adam = keras.optimizers.Adam(lr=0.001, decay=0.0, clipnorm=5.)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1,
                              patience=20, min_lr=0.00001, verbose=True)


# compile the model with binary_crossentropy loss for multilabel
model.compile(optimizer=adam, loss='binary_crossentropy')

model_name = 'model' + run + '.h5'
checkpoint = ModelCheckpoint(model_name, monitor='val_loss', verbose=1, save_best_only=True, mode='min')

callbacks_list = [checkpoint, reduce_lr]


# look at the params before training
#model.summary()

In [15]:
### from keras.models import load_model
#model_name = 'model' + run + '.h5'
#model = load_model(model_name)
#adam = keras.optimizers.Adam(lr=0.001, decay=0.0, clipnorm=5.0)

#model.compile(optimizer=adam, loss='binary_crossentropy')

#checkpoint = ModelCheckpoint(model_name, monitor='val_loss', verbose=1, save_best_only=True, mode='min')
#callbacks_list = [checkpoint]


model.fit_generator(
    data.batches(), 
    steps_per_epoch=data.num_batches,                  # data.num_batches to train on full set 
    epochs=25
)
#model.save(model_name)

Epoch 1/25
1/1 [==============================] - 11s - loss: 0.1821
Epoch 2/25
1/1 [==============================] - 3s - loss: 0.1890
Epoch 3/25
1/1 [==============================] - 7s - loss: 0.1150
Epoch 4/25
1/1 [==============================] - 7s - loss: 0.1054
Epoch 5/25
1/1 [==============================] - 7s - loss: 0.1040
Epoch 6/25
1/1 [==============================] - 6s - loss: 0.0933
Epoch 7/25
1/1 [==============================] - 7s - loss: 0.0742
Epoch 8/25
1/1 [==============================] - 7s - loss: 0.0715
Epoch 9/25
1/1 [==============================] - 7s - loss: 0.0843
Epoch 10/25
1/1 [==============================] - 7s - loss: 0.0790
Epoch 11/25
1/1 [==============================] - 7s - loss: 0.0798
Epoch 12/25
1/1 [==============================] - 6s - loss: 0.0698
Epoch 13/25
1/1 [==============================] - 7s - loss: 0.0564
Epoch 14/25
1/1 [==============================] - 7s - loss: 0.0559
Epoch 15/25
1/1 [=========================

In [16]:
# instantiate model
#model = VGG19()
model = MobileNet(dropout=0.2)


# classifier with sigmoid activation for multilabel

adam = keras.optimizers.Adam(lr=0.0003, decay=0.0, clipnorm=5.)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1,
                              patience=20, min_lr=0.00001, verbose=True)


# compile the model with binary_crossentropy loss for multilabel
model.compile(optimizer=adam, loss='binary_crossentropy')

model_name = 'model' + run + '.h5'
checkpoint = ModelCheckpoint(model_name, monitor='val_loss', verbose=1, save_best_only=True, mode='min')

callbacks_list = [checkpoint, reduce_lr]

In [17]:
model.fit_generator(
    data.batches(), 
    steps_per_epoch=data.num_batches,                  # data.num_batches to train on full set 
    epochs=25
)

Epoch 1/25
1/1 [==============================] - 12s - loss: 0.1709
Epoch 2/25
1/1 [==============================] - 3s - loss: 0.1227
Epoch 3/25
1/1 [==============================] - 7s - loss: 0.0834
Epoch 4/25
1/1 [==============================] - 7s - loss: 0.0940
Epoch 5/25
1/1 [==============================] - 7s - loss: 0.1194
Epoch 6/25
1/1 [==============================] - 7s - loss: 0.0764
Epoch 7/25
1/1 [==============================] - 7s - loss: 0.0870
Epoch 8/25
1/1 [==============================] - 7s - loss: 0.1017
Epoch 9/25
1/1 [==============================] - 7s - loss: 0.0796
Epoch 10/25
1/1 [==============================] - 7s - loss: 0.0586
Epoch 11/25
1/1 [==============================] - 7s - loss: 0.0822
Epoch 12/25
1/1 [==============================] - 7s - loss: 0.0722
Epoch 13/25
1/1 [==============================] - 7s - loss: 0.0928
Epoch 14/25
1/1 [==============================] - 7s - loss: 0.0576
Epoch 15/25
1/1 [=========================

In [18]:
model = MobileNet(dropout=0.5)


# classifier with sigmoid activation for multilabel

adam = keras.optimizers.Adam(lr=0.001, decay=0.0, clipnorm=5.)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1,
                              patience=20, min_lr=0.00001, verbose=True)


# compile the model with binary_crossentropy loss for multilabel
model.compile(optimizer=adam, loss='binary_crossentropy')

model_name = 'model' + run + '.h5'
checkpoint = ModelCheckpoint(model_name, monitor='val_loss', verbose=1, save_best_only=True, mode='min')

callbacks_list = [checkpoint, reduce_lr]  

In [19]:
model.fit_generator(
    data.batches(), 
    steps_per_epoch=data.num_batches,                  # data.num_batches to train on full set 
    epochs=25
)

Epoch 1/25
1/1 [==============================] - 14s - loss: 0.1825
Epoch 2/25
1/1 [==============================] - 3s - loss: 0.1466
Epoch 3/25
1/1 [==============================] - 8s - loss: 0.1108
Epoch 4/25
1/1 [==============================] - 8s - loss: 0.0969
Epoch 5/25
1/1 [==============================] - 8s - loss: 0.0763
Epoch 6/25
1/1 [==============================] - 8s - loss: 0.1171
Epoch 7/25
1/1 [==============================] - 8s - loss: 0.1140
Epoch 8/25
1/1 [==============================] - 8s - loss: 0.0935
Epoch 9/25
1/1 [==============================] - 8s - loss: 0.0684
Epoch 10/25
1/1 [==============================] - 8s - loss: 0.0887
Epoch 11/25
1/1 [==============================] - 8s - loss: 0.0655
Epoch 12/25
1/1 [==============================] - 8s - loss: 0.0785
Epoch 13/25
1/1 [==============================] - 8s - loss: 0.0584
Epoch 14/25
1/1 [==============================] - 7s - loss: 0.0697
Epoch 15/25
1/1 [=========================

In [20]:
model = MobileNet(dropout=0.5)


# classifier with sigmoid activation for multilabel

adam = keras.optimizers.Adam(lr=0.0003, decay=0.0, clipnorm=5.)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1,
                              patience=20, min_lr=0.00001, verbose=True)


# compile the model with binary_crossentropy loss for multilabel
model.compile(optimizer=adam, loss='binary_crossentropy')

model_name = 'model' + run + '.h5'
checkpoint = ModelCheckpoint(model_name, monitor='val_loss', verbose=1, save_best_only=True, mode='min')

callbacks_list = [checkpoint, reduce_lr] 

In [21]:
model.fit_generator(
    data.batches(), 
    steps_per_epoch=data.num_batches,                  # data.num_batches to train on full set 
    epochs=25
)

Epoch 1/25
1/1 [==============================] - 16s - loss: 0.1779
Epoch 2/25
1/1 [==============================] - 2s - loss: 0.1067
Epoch 3/25
1/1 [==============================] - 8s - loss: 0.1025
Epoch 4/25
1/1 [==============================] - 8s - loss: 0.1508
Epoch 5/25
1/1 [==============================] - 8s - loss: 0.0808
Epoch 6/25
1/1 [==============================] - 8s - loss: 0.0891
Epoch 7/25
1/1 [==============================] - 8s - loss: 0.0876
Epoch 8/25
1/1 [==============================] - 8s - loss: 0.0769
Epoch 9/25
1/1 [==============================] - 8s - loss: 0.0778
Epoch 10/25
1/1 [==============================] - 8s - loss: 0.0583
Epoch 11/25
1/1 [==============================] - 8s - loss: 0.0857
Epoch 12/25
1/1 [==============================] - 8s - loss: 0.1211
Epoch 13/25
1/1 [==============================] - 8s - loss: 0.0718
Epoch 14/25
1/1 [==============================] - 8s - loss: 0.0582
Epoch 15/25
1/1 [=========================

Exception in thread Thread-6:
Traceback (most recent call last):
  File "/home/cs231n/myVE35/lib/python3.5/site-packages/skvideo/io/ffmpeg.py", line 267, in _read_frame_data
    assert len(arr) == framesize
AssertionError

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/lib/python3.5/threading.py", line 914, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.5/threading.py", line 862, in run
    self._target(*self._args, **self._kwargs)
  File "/home/cs231n/myVE35/lib/python3.5/site-packages/keras/utils/data_utils.py", line 568, in data_generator_task
    generator_output = next(self._generator)
  File "/home/ninja2/Primatrix/primatrix_dataset_utils_overfit.py", line 148, in batches
    verbose=verbose,y=y)
  File "/home/ninja2/Primatrix/primatrix_dataset_utils_overfit.py", line 267, in _get_video_batch
    video = skv.vread(filepath, as_grey=as_grey)
  File "/home/cs231n/myVE35/lib/python3.5/site-packages/sk

KeyboardInterrupt: 

In [22]:
model = MobileNet(dropout=0.2)


# classifier with sigmoid activation for multilabel

adam = keras.optimizers.Adam(lr=0.003, decay=0.0, clipnorm=5.)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1,
                              patience=20, min_lr=0.00001, verbose=True)


# compile the model with binary_crossentropy loss for multilabel
model.compile(optimizer=adam, loss='binary_crossentropy')

model_name = 'model' + run + '.h5'
checkpoint = ModelCheckpoint(model_name, monitor='val_loss', verbose=1, save_best_only=True, mode='min')

callbacks_list = [checkpoint, reduce_lr]

In [23]:
model.fit_generator(
    data.batches(), 
    steps_per_epoch=data.num_batches,                  # data.num_batches to train on full set 
    epochs=25
)

Epoch 1/25
1/1 [==============================] - 17s - loss: 0.1899
Epoch 2/25
1/1 [==============================] - 2s - loss: 0.2098
Epoch 3/25
1/1 [==============================] - 9s - loss: 0.0923
Epoch 4/25
1/1 [==============================] - 9s - loss: 0.1083
Epoch 5/25
1/1 [==============================] - 9s - loss: 0.1013
Epoch 6/25
1/1 [==============================] - 9s - loss: 0.0829
Epoch 7/25
1/1 [==============================] - 9s - loss: 0.0867
Epoch 8/25
1/1 [==============================] - 9s - loss: 0.0972
Epoch 9/25
1/1 [==============================] - 9s - loss: 0.0800
Epoch 10/25
1/1 [==============================] - 9s - loss: 0.0943
Epoch 11/25
1/1 [==============================] - 9s - loss: 0.0833
Epoch 12/25
1/1 [==============================] - 9s - loss: 0.0862
Epoch 13/25
1/1 [==============================] - 9s - loss: 0.0725
Epoch 14/25
1/1 [==============================] - 9s - loss: 0.0595
Epoch 15/25
1/1 [=========================

In [ ]:
model = MobileNet(dropout=0.5)


# classifier with sigmoid activation for multilabel

adam = keras.optimizers.Adam(lr=0.003, decay=0.0, clipnorm=5.)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1,
                              patience=20, min_lr=0.00001, verbose=True)


# compile the model with binary_crossentropy loss for multilabel
model.compile(optimizer=adam, loss='binary_crossentropy')

model_name = 'model' + run + '.h5'
checkpoint = ModelCheckpoint(model_name, monitor='val_loss', verbose=1, save_best_only=True, mode='min')

callbacks_list = [checkpoint, reduce_lr]

In [ ]:
model.fit_generator(
    data.batches(), 
    steps_per_epoch=data.num_batches,                  # data.num_batches to train on full set 
    epochs=25
)